# INITIAL SETUP

In [1]:
# Import dependencies
import random
import copy
import numpy as np
import time

In [2]:
# Start the full timer
boot_start = time.time()

In [3]:
# custom functions

# Defining dictionary layout of decks
def new_deck():
    return {"suits": [],
            "values": [],
            "ids": []}

# Takes an input, and randomly selects an index from the original, to one by one assemble a shuffled deck
def randomized_shuffling(deck):
    # Create a copy of the original deck to work with
    bin_deck = copy.deepcopy(deck)

    # Create a new deck to store the shuffled cards
    master_deck = new_deck()

    # Randomize the deck
    while len(bin_deck["suits"]) > 0:
        # Pick a random index
        random_index = random.randint(0, len(bin_deck["suits"]) - 1)

        # Add the card at the random index to the master deck
        for key in ["suits", "values", "ids"]:
            master_deck[key].append(bin_deck[key].pop(random_index))

    return master_deck

# Takes an input, and shuffles is in an alternating pattern
def interlaced_shuffling(deck):
    # Initial Setup
    temp_deck = copy.deepcopy(deck)

    # Randomize the deck a random number of times
    shuffle_count = random.randint(10, 20)
    for _ in range(shuffle_count):
        # Create a temporary deck to store the shuffled cards
        master_deck = new_deck()

        # Interlace shuffle
        half = len(temp_deck["suits"]) // 2
        for card in range(half):
            for key in ["suits", "values", "ids"]:
                master_deck[key].append(temp_deck[key][half + card])
                master_deck[key].append(temp_deck[key][card])

    return master_deck

# Takes 2 inputs, and seperates them accross 52 bins in a random order before reassembling by bin
def rotary_shuffling(master_deck, discard_deck):
    # Port and merge to temp dictionary
    temp_deck = new_deck()
    for key in temp_deck:
        temp_deck[key].extend(discard_deck[key])
        temp_deck[key].extend(master_deck[key])

    # Resets
    master_deck = new_deck()
    discard_deck = new_deck()

    # Rotary shuffler reset / setup
    rotary_shuffler = {}
    slot_bins = list(range(52))
    open_bins = list(range(52))

    for id in range(52):
        # Building out dictionary with 52 bins
        slot_name = f"Bin {id}"
        rotary_shuffler[slot_name] = new_deck().copy()

    # Loop until all cards are binned
    while temp_deck["suits"]:
        # Randomized target
        current_slot = random.choice(open_bins)
        slot_name = f"Bin {current_slot}"

        # Checking if bin has room, and transferring values if able
        if len(rotary_shuffler[slot_name]["suits"]) <= 8:
            for key in ["suits", "values", "ids"]:
                rotary_shuffler[slot_name][key].append(temp_deck[key].pop(0))
            if len(rotary_shuffler[slot_name]["suits"]) == 8:
                open_bins.remove(current_slot)

    # Dump bins to new master_deck
    opposite_slot = random.choice(slot_bins) + 25

    while slot_bins:
        # Adjustments to stay within correct index range
        opposite_slot %= 52
        slot_name = f"Bin {opposite_slot}"

        # Transferring values back to master_deck
        for key in ["suits", "values", "ids"]:
            master_deck[key].extend(rotary_shuffler[slot_name][key])

        del rotary_shuffler[slot_name]
        slot_bins.remove(opposite_slot)

        # Move to next bin
        opposite_slot += 1

    # Return the shuffled master_deck and the emptied/reset discard_deck
    return master_deck, discard_deck

In [4]:
#Inital Deck build
# Unicode icons
spade = '\u2660'
club = '\u2663'
heart = '\u2665'
diamond = '\u2666'

#setting up unique features
suits = [spade,diamond,club,heart]
faces = ["A","2","3","4","5","6","7","8","9","10","J","Q","K"]
values = [11,2,3,4,5,6,7,8,9,10,10,10,10]

# initial setup
raw_deck = {"suits": [],
            "values": []}

# loop to initalize all unique combinations in both bins
for suit_index,suit in enumerate(suits):
    # forward iteration for first 2 suits
    if suit_index < 2:
        for face_index, face in enumerate(faces):
            raw_deck["suits"] += [suit + face]
            raw_deck["values"] += [values[face_index]]
    # reverse iteration for last 2 suits
    else:
        for face_index, face in enumerate(reversed(faces)):
            raw_deck["suits"] += [suit + face]
            raw_deck["values"] += [values[-face_index-1]]
            
# Now building a deck of 8 decks in new deck order
raw_master = new_deck()

# loop to make master deck using 8 raw decks
for deck in range(8):
    raw_master["suits"] += raw_deck["suits"]
    raw_master["values"] += raw_deck["values"]
    
# loop to create unique ids for each of the cards made in the master deck
for id in range(len(raw_master["suits"])):
    raw_master["ids"] += [id]

In [5]:
# doing a random shuffle first
master_deck = randomized_shuffling(raw_master)
# then doing an interlaced shuffle to further randomize
master_deck = interlaced_shuffling(master_deck)

# initializing discard deck
discard_deck = new_deck()

# INITIAL TESTING

In [6]:
# test prints
print(len(master_deck["suits"]))
print(len(master_deck["values"]))
print(len(master_deck["ids"]))
print(master_deck["suits"][0:9])
print(master_deck["values"][0:9])
print(master_deck["ids"][0:9])

416
416
416
['♥10', '♦K', '♦Q', '♣2', '♦9', '♥K', '♥A', '♥Q', '♣10']
[10, 10, 10, 2, 9, 10, 11, 10, 10]
[354, 389, 284, 297, 177, 403, 51, 404, 341]


In [7]:
#Re-Testing random pairing for accuracy
max_range = len(master_deck["suits"])-2

for X in range(10):
    
    first = random.randint(0,max_range)
    second = first+1

    # print test
    print(f"Hand: {X+1}")
    print("   First Card:",master_deck["suits"][first])
    print("   Second Card:",master_deck["suits"][second])
    print("   Hand Value:", (master_deck["values"][first] + master_deck["values"][second]))

Hand: 1
   First Card: ♦2
   Second Card: ♦7
   Hand Value: 9
Hand: 2
   First Card: ♦A
   Second Card: ♦8
   Hand Value: 19
Hand: 3
   First Card: ♥10
   Second Card: ♠3
   Hand Value: 13
Hand: 4
   First Card: ♦2
   Second Card: ♣5
   Hand Value: 7
Hand: 5
   First Card: ♠6
   Second Card: ♣J
   Hand Value: 16
Hand: 6
   First Card: ♦3
   Second Card: ♣2
   Hand Value: 5
Hand: 7
   First Card: ♣6
   Second Card: ♦8
   Hand Value: 14
Hand: 8
   First Card: ♦9
   Second Card: ♦K
   Hand Value: 19
Hand: 9
   First Card: ♣10
   Second Card: ♣5
   Hand Value: 15
Hand: 10
   First Card: ♣2
   Second Card: ♦9
   Hand Value: 11


# SIMULATION

In [8]:
win_rates = {"PJDL":0, #Player blackjack, dealer lost
             "PJDB":0, #Player blackjack, dealer bust
             "PLDJ":0, #Player lost, dealer blackjack
             "PBDJ":0, #Player bust, dealer blackjack
             "PJDJ":0, #Push, each with blackjacks
             "PWDL":0, #Player wins, dealer lost
             "PWDB":0, #Player wins, dealer bust
             "PLDW":0, #Player loss, dealer wins
             "PBDW":0, #Player bust, dealer wins
             "PBDB":0, #Player bust, dealer bust
             "PpDp":0, #Standard Push
             "Errors":0} #Error Check

In [9]:
# randomly picks a shuffle trigger between the first and last 50 cards
cut_target = random.randint(49,(len(master_deck['suits']) - 51))

# Initial Setup
shuffles = 0
games = 0
dealer = new_deck()
player = new_deck()

# Error finding
error_a = 0
error_b = 0
error_c = 0
error_d = 0
error_e = 0

# Start the deal timer
deal_start = time.time()

# loop for X rounds
for rounds in range(1):
    #_____________________________________________________
    # Shuffle if cut target is reached
    
    if len(master_deck['suits']) <= cut_target:
        master_deck, discard_deck = rotary_shuffling(master_deck, discard_deck)
        shuffles += 1
        # generate new cut target
        cut_target = random.randint(49,(len(master_deck['suits']) - 51))
    #_____________________________________________________
    # Initial Draws
    
    for deal in range(2):
        for key in ["suits", "values", "ids"]:
            dealer[key].append(master_deck[key].pop(0))
        for key in ["suits", "values", "ids"]:
            player[key].append(master_deck[key].pop(0))
            
    #_____________________________________________________      
    # Check for instant loss/push condition, Skips to results if met
    
    if sum(dealer['values']) != 21:
        
        #_____________________________________________________
        #Player Play

        while sum(player['values']) <= 16:
            for key in ["suits", "values", "ids"]:
                player[key].append(master_deck[key].pop(0))

        #_____________________________________________________
        #Dealer Play

        # draw to reach limit
        while sum(dealer['values']) <= 17:
            for key in ["suits", "values", "ids"]:
                dealer[key].append(master_deck[key].pop(0))
                
    #_____________________________________________________
    
    # Round Results
    
    # Player blackjack
    if sum(player['values']) == 21 and len(player['values']) == 2:
        if sum(dealer['values']) == 21:
            if len(dealer['values']) == 2:
                win_rates["PJDJ"] += 1  # Push, each with blackjacks
            else:
                win_rates["PpDp"] += 1  # Standard Push
        elif sum(dealer['values']) > 21:
            win_rates["PJDB"] += 1  # Player blackjack, dealer bust
        elif sum(dealer['values']) < 21:
            win_rates["PJDL"] += 1  # Player blackjack, dealer lost
        else:
            win_rates["Errors"] += 1  # Error bin
            error_a += 1

    # Dealer blackjack        
    elif sum(dealer['values']) == 21 and len(dealer['values']) == 2:
        if sum(player['values']) != 21:
            win_rates["PLDJ"] += 1  # Player lost, dealer blackjack
        else:
            win_rates["Errors"] += 1  # Error bin
            error_b += 1

    # Player valid        
    elif sum(player['values']) <= 21: 
        if sum(dealer['values']) <= 21:
            if sum(player['values']) > sum(dealer['values']):
                win_rates["PWDL"] += 1  # Player wins, dealer lost
            elif sum(player['values']) < sum(dealer['values']):
                win_rates["PLDW"] += 1  # Player loss, dealer wins
            elif sum(dealer['values']) > 21:
                win_rates["PWDB"] += 1  # Player wins, dealer bust
            elif sum(player['values']) == sum(dealer['values']):
                win_rates["PpDp"] += 1  # Standard Push
            else:
                win_rates["Errors"] += 1  # Error bin
                error_c += 1
        else:
            win_rates["PWDB"] += 1  # Player wins, dealer bust
            
    # Player busts
    else:
        if sum(dealer['values']) <= 21:  # Dealer valid
            win_rates["PBDW"] += 1  # Player bust, dealer wins
        elif sum(dealer['values']) > 21:
            win_rates["PBDB"] += 1  # Player bust, dealer bust
        else:
            win_rates["Errors"] += 1  # Error bin
            error_d += 1

    # Discard Transfer
    for key, value in dealer.items():
        if key in discard_deck:
            discard_deck[key] += value

    for key, value in player.items():
        if key in discard_deck:
            discard_deck[key] += value

    print(f"{player}")
    # Bin resets
    dealer = new_deck()
    player = new_deck()

    games += 1
    
# End the deal timer
deal_end = time.time()

{'suits': ['♦K', '♣2', '♦9'], 'values': [10, 2, 9], 'ids': [389, 297, 177]}


In [10]:
# verifying count (416 = correct)
print(len(discard_deck['suits'])+len(master_deck['suits']))

416


# RESULTS

In [11]:
print(f"Number of games = {games}")
print(f"Number of shuffles = {shuffles}")

Number of games = 1
Number of shuffles = 0


In [12]:
# Calculating sum of results
total_plays = 0
for key in win_rates:
    total_plays += win_rates[key]

print(f"Player blackjack, dealer lost = {win_rates['PJDL']}")
print(f"Player blackjack, dealer bust = {win_rates['PJDB']}")
print(f"Player lost, dealer blackjack = {win_rates['PLDJ']}")
print(f"Player bust, dealer blackjack = {win_rates['PBDJ']}")
print(f"Push, each with blackjacks = {win_rates['PJDJ']}")
print(f"Player wins, dealer lost = {win_rates['PWDL']}")
print(f"Player wins, dealer bust = {win_rates['PWDB']}")
print(f"Player loss, dealer wins = {win_rates['PLDW']}")
print(f"Player bust, dealer wins = {win_rates['PBDW']}")
print(f"Player bust, dealer bust = {win_rates['PBDB']}")
print(f"Standard Push = {win_rates['PpDp']}")
print(f"Error Count = {win_rates['Errors']}")
print("") # space
print(f"Total results = {total_plays}")

Player blackjack, dealer lost = 0
Player blackjack, dealer bust = 0
Player lost, dealer blackjack = 0
Player bust, dealer blackjack = 0
Push, each with blackjacks = 0
Player wins, dealer lost = 1
Player wins, dealer bust = 0
Player loss, dealer wins = 0
Player bust, dealer wins = 0
Player bust, dealer bust = 0
Standard Push = 0
Error Count = 0

Total results = 1


In [13]:
win_rate = (win_rates['PJDL']+
            win_rates['PJDB']+
            win_rates['PWDL']+
            win_rates['PWDB'])
print(f" Win Rate = {(win_rate/total_plays)*100}%")

 Win Rate = 100.0%


In [14]:
print(error_a)
print(error_b)
print(error_c)
print(error_d)
print(error_e)

0
0
0
0
0


In [15]:
# End the full timer
boot_end = time.time()

# Calculate the elapsed times
deal_time = deal_end - deal_start
boot_time = boot_end - boot_start

# Print the elapsed time
print(f"{games} deals took: {deal_time} seconds")
print(f"Full runtime: {boot_time} seconds")

1 deals took: 0.0 seconds
Full runtime: 0.20363736152648926 seconds


In [16]:
#BJ21e Performance:
#1m deals = 25.75 seconds
#Full run = 25.96 seconds